# Test fetch account

In [ ]:
from deepdiff import DeepDiff
from laminhub_rest.connector import get_access_token
from laminhub_rest.main import client
from laminhub_rest.routers.account import (
    get_account_by_handle,
    get_account_by_id,
    get_account_instances,
)
from laminhub_rest.connector import connect_hub
from laminhub_rest.utils._test import (
    create_test_auth,
    create_test_account,
    create_test_storage,
    create_test_instance,
    add_test_collaborator,
)
from laminhub_rest.core.account import delete_account
from laminhub_rest.core.instance import delete_instance
from laminhub_rest._clean_ci import clean_ci

## Parameterize 

In [ ]:
hub = connect_hub()

# Create account
auth = create_test_auth()
access_token = auth["access_token"]
account = create_test_account(handle=auth["handle"], access_token=access_token)
account_id = account["id"]
account_handle = account["handle"]

# Create storage
storage = create_test_storage(access_token=access_token)

# Create instance
instance = create_test_instance(storage_id=storage["id"], access_token=access_token)
instance_id = instance["id"]

# Add account as an admin member
instance_collaborator = add_test_collaborator(
    instance_id=instance_id,
    account_id=account_id,
    role="admin",
    access_token=access_token,
)

## Fetch account by id

### Call `get_account_by_id`

In [ ]:
account = get_account_by_id(account_id)
assert str(account) == str(account)

### Get method on `/account/{id}` route

In [ ]:
response = client.get(f"/account/{account_id}")
assert str(response.json()) == str(account)

## Fetch account by handle

### Call `get_account_by_handle`

In [ ]:
account = get_account_by_handle(account_handle)
assert str(account) == str(account)

### Get method on `/account/{handle}` route

In [ ]:
response = client.get(f"/account/handle/{account_handle}")
assert str(response.json()) == str(account)

## Fetch account's instances

In [ ]:
account_instances_expected = [
    {
        **instance,
        "storage": {"root": storage["root"]},
        "account": {"handle": account_handle, "id": account_id},
    }
]

### Call `get_account_instances`

In [ ]:
instances = get_account_instances(account_handle, False, f"Bearer {access_token}")
assert DeepDiff(instances, account_instances_expected, ignore_order=True) == {}

### Get method on `/account/{id}` route

In [ ]:
response = client.get(
    f"/account/resources/instances/{account_handle}",
    headers={"authentication": f"Bearer {access_token}"},
)
assert str(response.json()) == str(account_instances_expected)

## Clean up

In [ ]:
delete_account(account_handle, access_token)
delete_instance(owner=account_handle, name=instance["name"], _access_token=access_token)

In [ ]:
clean_ci()